In [116]:
import pandas as pd
import numpy as np
from scipy.spatial import distance

In [117]:
import sys
sys.path.insert(1, '/scratch/cinthiasouza/mv-text-summarizer')

import itertools

In [118]:
%load_ext autoreload
%autoreload 2


from src import summarization


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [119]:
section='introduction'

In [120]:
import pickle

file = open("models_features_weights/predictions_proba.pkl",'rb')
predicitons = pickle.load(file)

In [121]:
df = pd.read_csv("dataset/embed_bert_{}_test.csv".format(section))

In [122]:
with open('dataset/dataset_{}.pkl'.format('features'), 'rb') as fp:
    dataset = pickle.load(fp)

In [123]:
predictions = [i[1] for i in  predicitons['introduction']['rf']]

In [124]:
df['sentences'] = dataset['introduction'][5]['sentences'].reset_index(drop=True)

In [125]:
df['articles'] = dataset['introduction'][5]['articles'].reset_index(drop=True)

In [126]:
df['predictions'] = predictions

In [127]:
columns = list(range(0, 383))
columns = list(map(str, columns))

In [128]:
grouped = df.groupby("articles")

In [129]:
def rerank_mmr(grouped, k=3, max_sentences=10, alpha=0.8):

    D = []
    summaries = {'articles':[], 're-rank':[]}

    for idx, group in grouped:

        D = []

        aux = group.sort_values("predictions").reset_index(drop=True)

        n_sentences = len(aux)
        n_sentences = max_sentences if n_sentences > max_sentences else n_sentences

        D.append(0)

        if n_sentences <=k:
            summaries.append(" ".join(group.loc[[i for i in range(n_sentences)]]['sentences'].tolist()))
        else:

            while len(D) < k:

                distances = []

                for i in  D:

                    for j in range(1, n_sentences):

                        if not j in D:

                            vi = aux.loc[i][columns]
                            vj = aux.loc[j][columns]

                            d = distance.cosine(vi, vj)
                            new_rank = alpha * aux.loc[j]['predictions'] + (1 - alpha) * d

                            distances.append((i, j, aux.loc[j]['predictions'], d, new_rank))

                distances.sort(key=lambda tup: tup[4], reverse=True)
                D.append(distances[0][1])

            summaries['re-rank'].append(" ".join(aux.loc[D]['sentences'].tolist()))
            summaries['articles'].append(idx)
            
    return summaries

In [130]:
references_df = pd.read_csv("dataset/references_df.csv")

In [131]:
summaries = rerank_mmr(grouped, k=3, max_sentences=6, alpha=0.9)

In [132]:
summaries_df = pd.DataFrame(summaries)

In [133]:
summaries_df = summaries_df.merge(references_df, on='articles')

In [134]:
summaries_df

,articles,re-rank,references
0,PMC2066182.json,Few studies have examined the long term conseq...,context: evidence suggests that babies' fat ma...
1,PMC2488407.json,"In humans, this region continues developing th...",summary information on the development and fun...
2,PMC2556985.json,Adolescence is a period of significant develop...,"the present study examined ethnic, gender and ..."
3,PMC2561942.json,Findings for mood problems appear less consist...,adenotonsillectomy (at) is among the most comm...
4,PMC2562256.json,The results of their studies were dependent on...,the performance of high sensitivity x ray imag...
...,...,...,...
919,PMC6980698.json,"In addition, we now understand that even thoug...",abnormal tau hyperphosphorylation and its aggr...
920,PMC6981036.json,Morning chronotype may be related to successfu...,research indicates that sleep duration and qua...
921,PMC6986562.json,". Thus, we build on this research through the ...",background: as the prevalence of depression is...
922,PMC6990458.json,This neglect is somewhat surprising. As a resu...,beats are among the basic units of perceptual ...


In [136]:
metrics=['rouge-1', 'rouge-2', 'rouge-l']

In [137]:
result = summarization.evaluate_summariesv2(summaries_df, ['re-rank'],metrics)

In [138]:
result.describe()

,re-rank_rouge-1,re-rank_rouge-2,re-rank_rouge-l
count,924.000000,924.000000,924.000000
mean,0.219171,0.049569,0.179524
std,0.074985,0.041801,0.047804
min,0.069444,0.000000,0.084969
25%,0.165297,0.020961,0.146585
50%,0.210918,0.038873,0.171728
75%,0.264495,0.066384,0.204730
max,0.566802,0.359184,0.498928
